In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.algs.dqn import *

In [2]:
# GridWorld (debugging)
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [3]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [4]:
env.show_model_policy(exp.policy.model)

[[ 0.         -0.00871241 -0.86779177 -1.90455866]
 [ 0.03664176 -0.78160262 -1.85105252 -2.86409497]
 [-0.7739231  -1.69984984 -2.71159482 -3.88958907]
 [-1.87910104 -2.71792126 -3.81940651 -4.641397  ]]
[['⟲' '←' '←' '←']
 ['↑' '←' '←' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '←']]
[[-0.12060058  0.         -0.03949767 -1.09724391]
 [-1.13142133  0.03082024 -0.99869597 -1.87450552]
 [-2.0682292  -1.0390892  -1.75991845 -3.07801366]
 [-2.9608047  -2.10979199 -3.08782887 -3.82114935]]
[['→' '⟲' '←' '←']
 ['→' '↑' '↑' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↑']]
[[-1.12599897  0.02298146  0.          0.09816235]
 [-2.01844263 -0.86556965  0.0285725  -0.84878677]
 [-2.88969374 -2.0006721  -0.86210424 -1.86063337]
 [-3.97291851 -2.80415511 -2.02262092 -2.86824298]]
[['→' '→' '⟲' '←']
 ['→' '→' '↑' '←']
 ['→' '↑' '↑' '↑']
 ['→' '→' '↑' '↑']]
[[-1.94503093 -1.02620101 -0.12376875  0.        ]
 [-2.97985363 -1.79962444 -0.76958978  0.09468484]
 [-3.61185622 -2.89031696 -1.76457143 -0.93290859]
 

In [5]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

tensor([[-1.6308, -1.9816, -0.0665, -1.8392]], device='cuda:0')
tensor([[-1.7335, -1.8371,  0.0861, -1.9192]], device='cuda:0')


In [6]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1722
Num with (s): 114. Num with (s, a): 74


In [0]:
env.visualize(exp.policy, steps=20)